In [16]:
import pandas as pd
from glob import glob
import numpy as np
from tqdm import tqdm
import json
from model_training_process import *
import requests

In [4]:
class_df = pd.read_csv("..\\tables\\corrected_class_df_pinyin_label_table.csv")
class_df[["pinyin", "class_label"]]

,pinyin,class_label
0,zhi,0
1,chi,0
2,shi,0
3,ri,0
4,zi,1
...,...,...
402,xun,20
403,yong,18
404,jiong,20
405,qiong,20


In [5]:
class_df['class_label_index'] = class_df.groupby('class_label').cumcount()
class_df[["pinyin", "class_label", "class_label_index"]]

,pinyin,class_label,class_label_index
0,zhi,0,0
1,chi,0,1
2,shi,0,2
3,ri,0,3
4,zi,1,0
...,...,...,...
402,xun,20,11
403,yong,18,4
404,jiong,20,12
405,qiong,20,13


In [6]:
npy_file_path_list = glob("..\\..\\data\\*EduVer\\*")
print(f"len(npy_file_path_list): {len(npy_file_path_list)}")

len(npy_file_path_list): 67173


In [7]:
selected_npy_file_path_list = list()
for npy_file_path in npy_file_path_list:
    pinyin = npy_file_path.split("_")[1]
    pinyin_non_accent = pinyin[:-1] if pinyin[-1].isdigit() else pinyin
    if pinyin_non_accent in list(class_df["pinyin"]):
        selected_npy_file_path_list.append(npy_file_path)
print(f"len(npy_file_path_selected): {len(selected_npy_file_path_list)}")

len(npy_file_path_selected): 66852


In [8]:
print(len(class_df.groupby("class_label").count().index))
class_df.groupby("class_label").count().index

21


Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20],
      dtype='int64', name='class_label')

In [9]:
dic = dict()

for i in range(len(class_df.groupby("class_label").count().index)):
    dic[f"class_{i}_mfcc_matrix_list"] = list()
    dic[f"class_{i}_class_label_index_list"] = list()

for npy_file_path in tqdm(selected_npy_file_path_list):
    pinyin = npy_file_path.split("_")[1]
    pinyin_non_accent = pinyin[:-1] if pinyin[-1].isdigit() else pinyin

    class_label = class_df.loc[class_df["pinyin"] == pinyin_non_accent, "class_label"].values[0]
    class_label_index = class_df.loc[class_df["pinyin"] == pinyin_non_accent, "class_label_index"].values[0]
    # print(f"{pinyin_non_accent}, {class_label}, {class_label_index}")

    mfcc_matrix = np.load(npy_file_path)
    dic[f"class_{class_label}_mfcc_matrix_list"].append(mfcc_matrix)
    dic[f"class_{class_label}_class_label_index_list"].append(class_label_index)

for i in range(len(class_df.groupby("class_label").count().index)):
    dic[f"class_{i}_mfcc_matrix_list"] = np.array(dic[f"class_{i}_mfcc_matrix_list"])
    dic[f"class_{i}_class_label_index_list"] = np.array(dic[f"class_{i}_class_label_index_list"])

for i in range(len(class_df.groupby("class_label").count().index)):
    print(f"class_{i}_mfcc_matrix_list.shape: {dic[f'class_{i}_mfcc_matrix_list'].shape}")
    print(f"class_{i}_class_label_index_list.shape: {dic[f'class_{i}_class_label_index_list'].shape}")

100%|██████████| 66852/66852 [00:25<00:00, 2631.93it/s]

class_0_mfcc_matrix_list.shape: (3133, 13, 44)
class_0_class_label_index_list.shape: (3133,)
class_1_mfcc_matrix_list.shape: (844, 13, 44)
class_1_class_label_index_list.shape: (844,)
class_2_mfcc_matrix_list.shape: (1322, 13, 44)
class_2_class_label_index_list.shape: (1322,)
class_3_mfcc_matrix_list.shape: (5333, 13, 44)
class_3_class_label_index_list.shape: (5333,)
class_4_mfcc_matrix_list.shape: (7255, 13, 44)
class_4_class_label_index_list.shape: (7255,)
class_5_mfcc_matrix_list.shape: (4499, 13, 44)
class_5_class_label_index_list.shape: (4499,)
class_6_mfcc_matrix_list.shape: (6060, 13, 44)
class_6_class_label_index_list.shape: (6060,)
class_7_mfcc_matrix_list.shape: (3050, 13, 44)
class_7_class_label_index_list.shape: (3050,)
class_8_mfcc_matrix_list.shape: (3167, 13, 44)
class_8_class_label_index_list.shape: (3167,)
class_9_mfcc_matrix_list.shape: (2396, 13, 44)
class_9_class_label_index_list.shape: (2396,)
class_10_mfcc_matrix_list.shape: (4181, 13, 44)
class_10_class_label_ind

In [10]:
best_params = json.load(open(f"..\\..\\grid_search\\method1_with_fake_data\\best_params.json", "r"))

learning_rate = best_params["learning_rate"]
num_filters = best_params["num_filters"]
dense_units = best_params["dense_unit"]
batch_size = best_params["batch_size"]
epochs = best_params["epochs"]

channel = 1
verbose = 2
num_classes = len(class_df.index)
test_size = 0.2
# mfcc_dim_1 = mfcc_matrix_list.shape[1]
# mfcc_dim_2 = mfcc_matrix_list.shape[2]
# print(f"mfcc_dim_1: {mfcc_dim_1}")
# print(f"mfcc_dim_2: {mfcc_dim_2}")

In [ ]:
model_train_info_list = list()

for i in tqdm(range(len(class_df.groupby("class_label").count().index))):
    X = dic[f"class_{i}_mfcc_matrix_list"]
    y = dic[f"class_{i}_class_label_index_list"]
    
    mfcc_dim_1 = X.shape[1]
    mfcc_dim_2 = X.shape[2]
    
    X_train, X_test, y_train, y_test = get_processed_data(
        X=X,
        y=y,
        num_classes=num_classes,
        mfcc_dim_1=mfcc_dim_1,
        mfcc_dim_2=mfcc_dim_2,
        channel=channel,
        test_size=test_size,
    )

    model = get_cnn_model(input_shape=(mfcc_dim_1, mfcc_dim_2, channel), num_classes=num_classes,
                          learning_rate=learning_rate, num_filters=num_filters, dense_units=dense_units)
    model_train_info = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=verbose,
                                 validation_data=(X_test, y_test))
    
    model_train_info_list.append(model_train_info)

  0%|          | 0/21 [00:00<?, ?it/s]

Epoch 1/250
20/20 - 1s - loss: 5.3856 - accuracy: 0.2706 - val_loss: 5.9146 - val_accuracy: 0.2663 - 1s/epoch - 55ms/step
Epoch 2/250
20/20 - 0s - loss: 4.1083 - accuracy: 0.5974 - val_loss: 5.6787 - val_accuracy: 0.0590 - 234ms/epoch - 12ms/step
Epoch 3/250
20/20 - 0s - loss: 3.0020 - accuracy: 0.6309 - val_loss: 5.3569 - val_accuracy: 0.0494 - 225ms/epoch - 11ms/step
Epoch 4/250
20/20 - 0s - loss: 2.0269 - accuracy: 0.6895 - val_loss: 4.8803 - val_accuracy: 0.0478 - 231ms/epoch - 12ms/step
Epoch 5/250
20/20 - 0s - loss: 1.4265 - accuracy: 0.7506 - val_loss: 4.3002 - val_accuracy: 0.2648 - 234ms/epoch - 12ms/step
Epoch 6/250
20/20 - 0s - loss: 1.0451 - accuracy: 0.7861 - val_loss: 3.7399 - val_accuracy: 0.2679 - 232ms/epoch - 12ms/step
Epoch 7/250
20/20 - 0s - loss: 0.8306 - accuracy: 0.7993 - val_loss: 3.1771 - val_accuracy: 0.2679 - 227ms/epoch - 11ms/step
Epoch 8/250
20/20 - 0s - loss: 0.7280 - accuracy: 0.8053 - val_loss: 2.6627 - val_accuracy: 0.2663 - 230ms/epoch - 12ms/step
Epo

  5%|▍         | 1/21 [00:58<19:39, 58.95s/it]

Epoch 1/250
6/6 - 1s - loss: 6.1235 - accuracy: 0.0207 - val_loss: 5.9936 - val_accuracy: 0.0769 - 802ms/epoch - 134ms/step
Epoch 2/250
6/6 - 0s - loss: 5.4212 - accuracy: 0.3600 - val_loss: 5.9457 - val_accuracy: 0.4320 - 63ms/epoch - 10ms/step
Epoch 3/250
6/6 - 0s - loss: 4.9555 - accuracy: 0.4963 - val_loss: 5.8702 - val_accuracy: 0.4379 - 78ms/epoch - 13ms/step
Epoch 4/250
6/6 - 0s - loss: 4.6748 - accuracy: 0.4681 - val_loss: 5.7657 - val_accuracy: 0.4438 - 78ms/epoch - 13ms/step
Epoch 5/250
6/6 - 0s - loss: 4.0923 - accuracy: 0.5200 - val_loss: 5.6389 - val_accuracy: 0.4438 - 78ms/epoch - 13ms/step
Epoch 6/250
6/6 - 0s - loss: 3.7614 - accuracy: 0.5230 - val_loss: 5.4905 - val_accuracy: 0.2781 - 64ms/epoch - 11ms/step
Epoch 7/250
6/6 - 0s - loss: 3.3940 - accuracy: 0.5556 - val_loss: 5.3336 - val_accuracy: 0.2781 - 77ms/epoch - 13ms/step
Epoch 8/250
6/6 - 0s - loss: 3.0808 - accuracy: 0.5526 - val_loss: 5.1714 - val_accuracy: 0.2781 - 78ms/epoch - 13ms/step
Epoch 9/250
6/6 - 0s -

In [ ]:
# DISCORD -> 設定 -> 整合 -> Webhook -> 新 Webhook > -> 複製 Webhook 網址
DISCORD_WEBHOOK_URL = "https://discord.com/api/webhooks/1287413088970346557/30gx7NdIfSxS1BRWk28IRkOHJeoET-ihIN_KAjYeXYkrpPeI0hBnE-68AHzhpTR4h3et"
requests.post(
    url=DISCORD_WEBHOOK_URL,
    data={"content": "cnn method2 layer2 with fake data模型訓練已完成!"}
)